In [1]:
import os
import airfrans as af
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn

In [2]:
import torch.optim as optim

In the notebook 3b we can see the architecture of a model. It is a MLP(Multiple Layer Perceptron) with encoder, hidden fully connected layers, and decoder.  
The input has 7 nodes for 7 inputs:  
- its position (two component in meter): 'x-position' and 'y-position'
- the inlet velocity (two components in meter per second): 'x-inlet_velocity' and 'y-inlet_velocity'
- the distance to the airfoil (one component in meter): distance_function
- the normals (two components in meter, set to 0 if the point is not on the airfoil): 'x-normals', 'y-normals'  

This means that each mesh point in each Simulation is a datapoint, so the input dimension is [7,1].  

The output of the network should be the size 4 for:  
- the velocity (two components in meter per second): 'x-velocity'and 'y-velocity'
- the pressure divided by the specific mass (one component in meter squared per second squared): 'pressure'
- the turbulent kinematic viscosity (one component in meter squared per second): 'turbulent_viscosity'
 




In [3]:
# If you already have the data downloaded, just specify the directory name:
directory_name = '../../TER/NOTEBOOKS/Dataset' # depends on where you have the data stored

In [4]:
dataset_list, dataset_name = af.dataset.load(root = directory_name, task = 'scarce', train = True)

Loading dataset (task: scarce, split: train): 100%|██████████| 200/200 [01:14<00:00,  2.69it/s]


In [5]:
first_simulation_name = dataset_name[0]

first_simulation = af.Simulation(root = directory_name, name = first_simulation_name, T = 298.15)

In [6]:
position = first_simulation.position
inlet_velocity = first_simulation.input_velocity
distance = first_simulation.sdf
normals = first_simulation.normals

velocity = first_simulation.velocity
pressure = first_simulation.pressure
nu_t = first_simulation.nu_t #kinematic turbulent viscosity

x_train = np.hstack((position, inlet_velocity, distance, normals))
y_train = np.hstack((velocity, pressure, nu_t))

In [7]:
for i in range(1,30):
    simulation_name = dataset_name[i]
    simulation = af.Simulation(root = directory_name, name = simulation_name, T = 298.15)
    position = simulation.position
    inlet_velocity = simulation.input_velocity
    distance = simulation.sdf
    normals = simulation.normals

    velocity = simulation.velocity
    pressure = simulation.pressure
    nu_t = simulation.nu_t

    x_train_add = np.hstack((position, inlet_velocity, distance, normals))
    y_train_add = np.hstack((velocity, pressure, nu_t))
    
    x_train = np.vstack((x_train, x_train_add))
    y_train = np.vstack((y_train, y_train_add))

In [8]:
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_train_torch = torch.from_numpy(x_train)
y_train_torch = torch.from_numpy(y_train)

In [9]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(7, 32),  
            nn.Tanh(),       
            nn.Linear(32, 32),  
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 4) 
        )

    def forward(self, x):
        return self.net(x)


Tanh maps values between -1 and 1, good for normalizing.

In [10]:
model = PINN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()  # Reset gradients
    y_pred = model(x_train_torch)  # Forward pass
    loss = criterion(y_pred, y_train_torch)  # Compute loss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [10/100], Loss: 684685.7500
Epoch [20/100], Loss: 684637.1875
Epoch [30/100], Loss: 684586.1875
Epoch [40/100], Loss: 684536.4375
Epoch [50/100], Loss: 684489.3125
Epoch [60/100], Loss: 684445.0625
Epoch [70/100], Loss: 684403.8750
Epoch [80/100], Loss: 684365.1250
Epoch [90/100], Loss: 684327.9375
Epoch [100/100], Loss: 684291.9375
